In [2]:
import re
import urllib.request
import urllib
from bs4 import BeautifulSoup
import json

from multiprocessing import freeze_support
freeze_support()

import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.webdriver import WebDriver
   
import time


class SamoletServiceProcessing:
  
    # def load_url_by_default(self):
    #    self.__urls = [
    #    "https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1"
    #    "https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1&offset=12&limit=12"
    #
    #    "https://samolet.ru/api_redesign/flats/200343/"
    #    ]

    def __generate_jsons(self):
        #for url_base in self.__urls:
        #    next_url = url_base
        #    while next_url:
        #        url = next_url
        #        next_url = ""
        #        print(url)
        #        #sessionid=kjw99ls6bcb4onofwazpcufjmwwa308x
        #        a_request = urllib.request.Request(url)
        #        #a_request.add_header("Cookie", "sessionid=kjw99ls6bcb4onofwazpcufjmwwa308x")
        #        #qrator_jsr=1701810731.684.adBf0Vn0XMh8G9sf-f67qh8uqbbmbh3a42siunqadovrbv0vp-00
        #        a_request.add_header("Cookie", "qrator_jsr=1701810731.684.adBf0Vn0XMh8G9sf-f67qh8uqbbmbh3a42siunqadovrbv0vp-00; Max-Age=300; Path=/;")
        #        with urllib.request.urlopen(a_request) as response:
        #        #with urllib.request.urlopen(url) as response:
        #            self.__page = response.read()
        #            jsons = json.loads(self.__page)
        #            yield (jsons, url)
        #            next_url = jsons.get("next", "")

        
        # формируем настройки для хрома
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
#        options.add_argument('--headless')  # отключает открытие браузера
        options.add_argument('--window-size=1280x1696')
        #options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')
        options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36') 

        options.add_argument('--disable-extensions')
        options.add_argument('--disable-application-cache')
        options.add_argument('--disable-gpu ')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-setuid-sandbox')
        options.add_argument('--disable-dev-shm-usage')




        
        #options.binary_location = '/usr/bin/chromium-browser'
        options.binary_location = '/usr/bin/chromedriver'

        chrome = uc.Chrome(options=options, headless=True,use_subprocess=False )
#        chrome = uc.Chrome(headless=True,use_subprocess=False )
        #chrome = uc.Chrome()
        
    
        #self.__list_dict = []
        #for url in self.__urls:
        #    for object_params in OnePage(url).list_dict():
        #        self.__list_dict.append(object_params)
        for url_base in self.__urls:
            next_url = url_base
            while next_url:
                url = next_url
                next_url = ""
                print(url)
 
                try:
                    # пробуем получить страницу каталога
                    chrome.get(url)
                    # даем время на загрузку страницы
                    time.sleep(3)
                    # получаем html страницы
                    page = chrome.page_source
                    # варим суп
                    #lenta_soup = BeautifulSoup(page, "lxml")
                    
                    print(page)
                    self.__page = page 
                    jsons = json.loads(self.__page)
                    yield (jsons, url)
                    next_url = jsons.get("next", "")

                except AttributeError as e:
                    print(f"Неудалось получить общий каталог, ошибка - {e}")


                break

    def process(self):
        self.__urls = [
            #"https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1",
            "https://alia.moscow/flats/?newTypeFlat=209",
        ]

        self.__list_dict = []
        for jsons, source_url in self.__generate_jsons():
            for e in jsons["results"]:
               
                el = {}
                el[
                    "title"
                ] = f"{e['project']} {e['building']} {str(e['floor_number'])} {str(e['area'])}"
                el["quantity"] = 1
                el["price"] = e["old_price_with_kitchen"]
                el["price_sale"] = e["filter_price"]
                el["datetime_create"] = "1970-01-01T00:00:00.00Z"
                el["brand"] = "samolet.ru"
                el["brand_url"] = "alia.moscow"
                el["category"] = "Новостройки"
                el["url"] = e["url"]
                el["image_url"] = e["plan"]
                el["source_url"] = source_url
                el["apartment_area"] = e["area"]
                el["apartment_floor"] = int(e["floor_number"])
                if e.get("total_floors"):
                    el["apartment_floors_total"] = int(e["total_floors"])
                else:
                    print("not have floors_total in =", source_url)
                    el["apartment_floors_total"] = None
                el["apartment_room"] = int(e["rooms"])
                el["apartment_ppm"] = int(e["ppm"])
                el["apartment_address"] = e["project"]
                #print(e["floor"])
                
                self.__list_dict.append(el)
            print(source_url)

    # def pp(self):
    #    return self.__list_dict

    # def __create_file_name_with_current_datetime(self):
    #    return 'lelev_fresh.json'

    # def save_in_file_with_current_datetime(self):
    #    json_string = json.dumps(self.__list_dict)
    #    file_name = self.__create_file_name_with_current_datetime()
    #    with open(file_name, 'w') as outfile:
    #        json.dump(json_string, outfile)

    def send_in_api(self):
        configuration = openapi_client.Configuration(host="http://absrent.ru:8000")

        with openapi_client.ApiClient(configuration) as api_client:
            api_instance = history_api.HistoryApi(api_client)
            for e in self.__list_dict:
                print(e)
        
            print("Count load object =", len(self.__list_dict))

In [ ]:
!ss -plnut

In [ ]:
#!pip install undetected_chromedriver

In [ ]:
#!pip install undetected-chromedriver

In [ ]:
!ls -la /home/jupyter-amtsu/.local/share/undetected_chromedriver/undetected_chromedriver

In [ ]:
tssp = SamoletServiceProcessing()
#tssp.load_url_by_default()
tssp.process()
tssp.send_in_api()

In [ ]:
options = uc.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")

#added new options since issues in Jan 2023 #743
options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--headless") #can enable or disable headless, up to your needs

driver = uc.Chrome(use_subprocess=True,options = options)

with driver:
    driver.get('https://www.101katok.ru')

In [7]:
#!curl https://alia.moscow/flats/?newTypeFlat=209

In [9]:
#driver = uc.Chrome()
#driver.get('https://alia.moscow/flats/?newTypeFlat=209')

In [ ]:
options = uc.ChromeOptions()
driver = uc.Chrome(options = options)
driver.get('http://101katok.ru')

In [ ]:
options = uc.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")

#added new options since issues in Jan 2023 #743
options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--headless") #can enable or disable headless, up to your needs

driver = uc.Chrome(use_subprocess=True,options = options)

driver.get('http://101katok.ru')

In [ ]:
options = uc.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")

#added new options since issues in Jan 2023 #743
options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--headless") #can enable or disable headless, up to your needs

driver = uc.Chrome(use_subprocess=True,options = options)

with driver:
    driver.get('https://101katok.ru')

In [ ]:


options = uc.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")

# get ua from config file
#ua = get_config_by_key("UA", "user_agent")

# update user-agent
#chrome_options.add_argument(f"--user-agent={ua}")
options.add_argument("--incognito")

options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--port=19222")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")

In [ ]:
driver = uc.Chrome()

In [ ]:
driver = uc.Chrome(use_subprocess=True)

#47565

In [ ]:
driver = uc.Chrome(use_subprocess=False)

In [ ]:
driver = uc.Chrome(options = options)

In [ ]:
driver = uc.Chrome(use_subprocess=True, options = options)

In [ ]:
#127.0.0.1:35573 

In [ ]:
driver = uc.Chrome(use_subprocess=False,options = options)

In [ ]:
driver.get('http://101katok.ru')

# пробуем на бьютифул чсоуп